In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

np.set_printoptions(suppress=True, precision=3)

sns.set(style='ticks', palette='Set2')
sns.despine()

In [6]:
import utils as U
import opt_runs as OR
import opt_model as OM
import decorated_options as Deco

The RedQueen demo expects data in the following format:

```
{
  "walls": {
    "1": [1,2,3,4,5], # These time-stamps do not contain posts by our user.
    "2": [1,4,5]
  },
  "broadcasts": {
    "redqueen": [1, 4],
    "poisson": [1, 3]
  }
}
```

However, this shifts the task of calculating performance to the JS library, which will require re-implementation and potential bugs. Hence, the performance data should be included with this data.

```
{
  "walls": {
    "1": [1,2,3,4,5],
    "2": [1,4,5]
  },
  "broadcasts": {
    "redqueen": {
        "post_times": [1, 4],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
     }, 
    "poisson": {
        "post_times": [1, 3],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
    },
  }
}
```

The `<metric>` could be `perf_top_{1,5,10}`, `avg_rank`, `r_2_int`, etc.

In [3]:
# Step 1: Create a SimOpts which will generate raw data

In [13]:
sim_opts_1_follower = OM.SimOpts(
    src_id=0,
    end_time=100,
    q_vec=np.array([1]),
    s=1.0,
    other_sources=[('Hawkes', {'src_id': 1, 'seed': 1, 'l_0': 10.0, 'alpha': 1.0, 'beta': 10.0})],
    sink_ids=[1000],
    edge_list=[(0, 1000), (1, 1000)]
)

In [1]:
# Step 2: Simulate the walls along with different strategies for our broadcaster

In [37]:
%%time
seed = 1
opt_mgr = sim_opts_1_follower.create_manager_with_opt(seed)
opt_mgr.run_dynamic()
opt_df = opt_mgr.state.get_dataframe()
num_opt_tweets = U.num_tweets_of(opt_df, broadcaster_id=0)
perf_opt = {
    'type': 'Opt',
    'seed': seed,
    'capacity': num_opt_tweets,
    's': sim_opts_1_follower.s
}
OR.add_perf(perf_opt, opt_df, sim_opts_1_follower)

CPU times: user 647 ms, sys: 0 ns, total: 647 ms
Wall time: 647 ms


In [38]:
%%time
seed = 9
poisson_mgr = sim_opts_1_follower.create_manager_with_poisson(seed, capacity=num_opt_tweets)
poisson_mgr.run_dynamic()
poisson_df = poisson_mgr.state.get_dataframe()
num_poisson_tweets = U.num_tweets_of(opt_df, broadcaster_id=0)
perf_poisson = {
    'type': 'Poisson',
    'seed': seed,
    'capacity': num_poisson_tweets,
    's': sim_opts_1_follower.s
}
OR.add_perf(perf_poisson, poisson_df, sim_opts_1_follower)

CPU times: user 887 ms, sys: 13.3 ms, total: 900 ms
Wall time: 894 ms


In [25]:
# Step 3: Calculate the performance and save JSON output

In [39]:
print('num_opt_tweets = {}, num_poisson_tweets = {}'
      .format(U.num_tweets_of(opt_df, 0), U.num_tweets_of(poisson_df, 0)))

num_opt_tweets = 253.0, num_poisson_tweets = 253.0


In [40]:
print('avg_rank_opt = {}, avg_rank_poisson = {}'
      .format(U.average_rank(opt_df, sim_opts=sim_opts_1_follower), 
              U.average_rank(poisson_df, sim_opts=sim_opts_1_follower)))

avg_rank_opt = 242.69749378358927, avg_rank_poisson = 441.09949791226717


In [41]:
perf_opt

{'avg_rank': 242.69749378358927,
 'capacity': 253.0,
 'num_events': 253,
 'r_2': 1094.6826309813168,
 's': 1.0,
 'seed': 1,
 'top_1': 22.176455535655961,
 'type': 'Opt',
 'world_events': 1143}

In [42]:
print('top_1_opt = {}, top_1_poisson = {}'
      .format(perf_opt['top_1'], perf_poisson['top_1']))

top_1_opt = 22.17645553565596, top_1_poisson = 17.02536436881686


The output JSON structure, for reference:

```
{
  "walls": {
    "1": [1,2,3,4,5],
    "2": [1,4,5]
  },
  "broadcasts": {
    "redqueen": {
        "post_times": [1, 4],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
     }, 
    "poisson": {
        "post_times": [1, 3],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
    },
  }
}
```

In [49]:
perf_opt

{'avg_rank': 242.69749378358927,
 'capacity': 253.0,
 'num_events': 253,
 'r_2': 1094.6826309813168,
 's': 1.0,
 'seed': 1,
 'top_1': 22.176455535655961,
 'type': 'Opt',
 'world_events': 1143}

In [52]:
tmp = U.rank_of_src_in_df(opt_df, 0).mean(1)
list(zip(tmp.index, tmp.values))

[(0.0, 0.0),
 (0.053960583725918547, 1.0),
 (0.053972021861782855, 2.0),
 (0.068400316900717725, 3.0),
 (0.068514698259360809, 0.0),
 (0.085970119680972359, 1.0),
 (0.12709353173139459, 2.0),
 (0.17169734817183441, 3.0),
 (0.19069969818871357, 4.0),
 (0.19291929424178314, 5.0),
 (0.23312246093329292, 6.0),
 (0.24478917725154159, 7.0),
 (0.26858121982556787, 0.0),
 (0.27258327155521556, 1.0),
 (0.27920109093163153, 2.0),
 (0.29235795047694624, 3.0),
 (0.29949165067347727, 4.0),
 (0.50880482131420557, 5.0),
 (0.52108235800377656, 0.0),
 (0.61825438895287987, 1.0),
 (0.72781056456887061, 2.0),
 (0.7295632736227291, 3.0),
 (1.1204880979923908, 4.0),
 (1.1532357147650858, 5.0),
 (1.1633641910135395, 6.0),
 (1.2679016110132999, 0.0),
 (1.3698296563050776, 1.0),
 (1.4026871923800308, 2.0),
 (1.4044721895355332, 3.0),
 (1.4244178076534662, 4.0),
 (1.4789980969039289, 5.0),
 (1.5503326378791771, 6.0),
 (1.6280308179858822, 7.0),
 (1.6377322316652543, 8.0),
 (1.6392121711900818, 0.0),
 (1.737091

In [55]:
@Deco.optioned('opts')
def perf_to_json(dfs, names, src_id, sink_ids):
    """Produce a dictionary which captures performance for the demo app."""        
    
    # Assumes that the walls across the data frames are the same.
    eg_df = dfs[0]
    
    walls = {}
    for sink_id in sink_ids:
        walls[sink_id] = eg_df[(eg_df.src_id != src_id) & (eg_df.sink_id == sink_id)].t.tolist()
    
    broadcasts = {}
    for df, name in zip(dfs, names):
        broadcasts[name] = {
            'post_times': df[df.src_id == src_id].t.unique().tolist(),
            'performance': {
                'avg_rank': list(zip(U.rank_of_src_in_df(df, src_id).mean(1)))
            }
        }
        
    return {
        'walls': walls,
        'broadcasts': broadcasts
    }


In [61]:
example_1 = perf_to_json([opt_df, poisson_df], ['redqueen', 'poisson'], opts=Deco.Options(**sim_opts_1_follower.get_dict()))

In [62]:
import json

In [65]:
with open('data/example1.json', 'w') as f:
    json.dump(example_1, f, indent=2)